In [ ]:
!pip install torch transformers sacremoses

In [ ]:
import torch
from transformers import *
import pdb
import operator
from collections import OrderedDict
import sys
import traceback
import argparse
import string


import logging

#DEFAULT_MODEL_PATH='bert-large-cased'
#DEFAULT_MODEL_PATH='bert-base-cased' #works best for names
#DEFAULT_MODEL_PATH='bert-base-uncased'
DEFAULT_MODEL_PATH='./'
DEFAULT_TO_LOWER=False
DEFAULT_TOP_K = 20
ACCRUE_THRESHOLD = 1

def init_model(model_path,to_lower):
    logging.basicConfig(level=logging.INFO)
    print("******* MODEL[path] is:",model_path," lower casing is set to:",to_lower)
    tokenizer = BertTokenizer.from_pretrained(model_path,do_lower_case=to_lower)
    model = BertForMaskedLM.from_pretrained(model_path)
    #tokenizer = RobertaTokenizer.from_pretrained(model_path,do_lower_case=to_lower)
    #model = RobertaForMaskedLM.from_pretrained(model_path)
    model.eval()
    return model,tokenizer

def get_sent(to_lower):
    print("Enter sentence (optionally including the special token [MASK] if tolower is set to False). Type q to quit")
    sent = input()
    if (sent == 'q'):
        return sent
    else:
        return  sent.lower() if to_lower else sent

def read_descs(file_name):
    ret_dict = {}
    with open(file_name) as fp:
        line = fp.readline().rstrip("\n")
        if (len(line) >= 1):
            ret_dict[line] = 1
        while line:
            line = fp.readline().rstrip("\n")
            if (len(line) >= 1):
                ret_dict[line] = 1
    return ret_dict



def perform_task(model,tokenizer,top_k,accrue_threshold,text,patched):
    text = '[CLS] ' + text + ' [SEP]' 
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

    print(tokenized_text)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])


    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)
        for i in range(len(tokenized_text)):
                #if (i != 0 and i != len(tokenized_text) - 1):
                #    continue
                results_dict = {}
                masked_index = i
                neighs_dict = {}
                if (patched):
                    for j in range(len(predictions[0][0][0,masked_index])):
                        if (float(predictions[0][0][0,masked_index][j].tolist()) > accrue_threshold):
                            tok = tokenizer.convert_ids_to_tokens([j])[0]
                            results_dict[tok] = float(predictions[0][0][0,masked_index][j].tolist())
                        tok = tokenizer.convert_ids_to_tokens([j])[0]
                        if (tok in tokenized_text):
                            neighs_dict[tok] = float(predictions[0][0][0,masked_index][j].tolist())
                else:
                    for j in range(len(predictions[0][0][masked_index])):
                        if (float(predictions[0][0][masked_index][j].tolist()) > accrue_threshold):
                            tok = tokenizer.convert_ids_to_tokens([j])[0]
                            results_dict[tok] = float(predictions[0][0][masked_index][j].tolist())
                        tok = tokenizer.convert_ids_to_tokens([j])[0]
                        if (tok in tokenized_text):
                            neighs_dict[tok] = float(predictions[0][0][masked_index][j].tolist())
                k = 0
                sorted_d = OrderedDict(sorted(results_dict.items(), key=lambda kv: kv[1], reverse=True))
                print("********* Top predictions for token: ",tokenized_text[i])
                for index in sorted_d:
                    if (index in string.punctuation or index.startswith('##') or len(index) == 1 or index.startswith('.') or index.startswith('[')):
                        continue
                    print(index,round(float(sorted_d[index]),4))
                    k += 1
                    if (k > top_k):
                        break
                print("********* Closest sentence neighbors in output to the token :  ",tokenized_text[i])
                sorted_d = OrderedDict(sorted(neighs_dict.items(), key=lambda kv: kv[1], reverse=True))
                for index in sorted_d:
                    if (index in string.punctuation or index.startswith('##') or len(index) == 1 or index.startswith('.') or index.startswith('[')):
                        continue
                    print(index,round(float(sorted_d[index]),4))
                print()
                print()
                #break




model = "bert-base-cased"
topk = DEFAULT_TOP_K
tolower=False
patched = False
threshold = ACCRUE_THRESHOLD
model,tokenizer = init_model(model,tolower)
text = "Aziz Khan is an [MASK]"
perform_task(model,tokenizer,topk,threshold,text,patched)